### **STEP-1**. AI Solution 및 Instance 등록을 위한 준비 작업
&#x1F600; **등록 할 AI Contents 의 experimental_plan.yaml 를 alo/config/ 에 준비해 둡니다.**

&#x1F600; **가상 환경을 만들어 두고, ipykernel 을 제작해 둡니다.**     

1. 본 jupyter notebook 실행 전에 alo/main.py 파일이 \
   존재하는 위치에서 아래 명령어들을 순차 실행합니다.
> http://collab.lge.com/main/pages/viewpage.action?pageId=2145631756 의 3번을 참고하여 가상환경 및 커널 생성합니다. \
> python main.py를 실행하여 학습 ~ 추론까지 1회 진행합니다. 

2. 본 jupyter notebook 에서, 위에서 생성한 ipykernel 을 선택 합니다. \
   가령 tcr이라는 이름의 가상환경을 만들었다면, 아래와 같이 선택합니다.

<div style="margin: 40px">
<img src="./image/ipykernel.png" width="400">
</div>

3. [임시] 본 솔루션 등록 쥬피터 노트북 최초 사용자는 고객지수 플랫폼 전체 Teams 방에 EP 계정 공유와 함께 cism-ws 권한 요청을 합니다.   

&#x1F600; **아래 STEP들을 하나씩 실행시키면서, <u>< 사용자 입력 ></u>이라고 주석 표기된 내용을 적절히 변경해주세요.**     

----

### **STEP-2**. AI Solution 이름 선택     

#### **STEP-2-1**. AI Conductor 시스템 URI 셋팅
&#x1F600; 로그인 요청 및 시스템 담당으로부터 사용 가능한 시스템 URI를 확인합니다. 
- 고객지수플랫폼 Development 
> URI: "https://aic-kic.aidxlge.com/"

> 시스템 URI 및 로그인 정보, ECR TAG, ICON FILE 명 등 사용자 입력부

In [ ]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
user_input ={
    # 로그인 정보: EP 정보로 입력해주세요 
    'LOGIN_ID': '', 
    'LOGIN_PW': '', 
    
    # 아래 정보들은 고객지수 플랫폼에선 변경할 필요 X 
    # workspace 이름
    'WORKSPACE_NAME': "cism-ws",
    
    # 시스템 URI
    'URI': "https://aic-web-kic.aidxlge.com/", 
    
    # ECR에 올라갈 컨테이너 URI TAG 
    # [참고] release-2.1.3 에선 latest만 허용 
    'ECR_TAG': 'latest', 
}
#----------------------------------------#

In [ ]:
#----------------------------------------#
#              사용자 입력               #
#----------------------------------------#
# 미리 생성해둔 conda 가상환경 이름을 입력합니다. 
VIRTUAL_ENV = 'test-env'
#----------------------------------------#> 시스템 로그인

In [ ]:
# 시각화, s3 용 모듈 install 
!pip install tabulate boto3 requests
# 이미 로컬 환경에서 python main.py 를 통해 한 번 솔루션을 실행한 상태에서, 솔루션의 dependency들을 requirements.txt로 만듭니다. 
!/home/jovyan/{VIRTUAL_ENV}/bin/pip list --format=freeze > requirements.txt

In [ ]:
# Generate RegisterUtils instance & login
from register_utils import RegisterUtils
registerer = RegisterUtils(user_input)

# AI Conductor Login
registerer.login()

> AI Solution 이름 설정 (사용자 입력)

In [ ]:
# AI Solution 이름 설정 (기존에 존재하는 중복 이름 허용X)
from IPython.display import display, HTML

# soltuion 이름 입력 받기 
display(HTML('<p style="font-size: 20px;"> < Create the name of AI Solution! > </p>'))
user_solution_name = input("- Enter the name you want to create: ")

# 기존에 존재하는 solution name과 겹치지 않는지 체크 
registerer.check_solution_name(user_solution_name)


#### **STEE-2-2**. AI Solution Name 을 AI Conductor 에 등록합니다. 
&#x1F600; 이름이 등록되면 본 jupyter 노트북 과정이 끝날 때까지 변경이 어려 울 수 있습니다. \
변경이 필요할 경우 <b> STEP-2-1 </b> 부터 다시 실행하여 주시기 바랍니다. 
 

> 기존에 workspace에 존재하는 solution 이름을 조회하여, 사용자가 입력하는 이름이 유효한지 (고유한지) 확인합니다.

In [ ]:
# 학습 용 solution_metadata.yaml 셋팅 
###################
pipeline = 'train'
###################
# check workspace (ECR, S3정보 셋팅까지 진행)
registerer.check_workspace() 
# 받아온 workspace 정보 기반으로 solution_metadata.yaml 셋팅 
registerer.set_yaml(pipeline = pipeline)

----

#### **STEP-3**. Train 용 Sample data, train artifacts, model, icon 등록

In [ ]:
# data 등록 
# s3 접근확인
registerer.s3_access_check('data')

# s3 데이터 업로드
# 이전에 이미 해당 s3 경로에 존재하던 데이터는 지워집니다.
registerer.s3_upload_data()

# artifacts 등록 
# s3 접근확인
registerer.s3_access_check('artifacts')

# s3 model, artifacts 업로드 
# 이전에 이미 해당 s3 경로에 존재하던 object는 지워집니다.
registerer.s3_upload_artifacts()

In [ ]:
html_content = registerer.display_icon()
display(HTML(html_content))

In [ ]:
#----------------------------------------#
#              사용자 입력               #
#----------------------------------------#
# 위에서 display 된 icon 중 하나를 선택해서 이름을 입력해주세요. 
SELECTED_ICON = 'ic_customer_diagnosis'
#----------------------------------------#> 
# UI에 표시될 아이콘 이미지를 선택합니다.
registerer.select_icon(SELECTED_ICON)

----

#### **STEP-4**. Train Docker Container 제작

> ALO 작업 폴더를 현재 노트북 경로로 가져옵니다. \
> DOCKERFILE을 셋팅합니다. 

In [ ]:
# ALO 작업 폴더 가져오기 
registerer.set_alo()

# DOCKERFILE setting
registerer.set_docker_contatiner()

> AWS ECR에 docker 등록을 위한 repository를 생성합니다. 

In [ ]:
## ECR 등록

## buildah login 실행 (docker in docker) - sudo 권한 필요 
tags = [
    "Key=Company,Value=LGE",
    "Key=Owner,Value=IC360",
    "Key=HQ,Value=CDO",
    "Key=Division,Value=CDO",
    "Key=Infra Region,Value=KIC",
    "Key=Service Mode,Value=DE",
    "Key=Cost Type,Value=COMPUTING",
    "Key=Project,Value=CIS",
    "Key=Sub Project,Value=CISM",
    "Key=System,Value=AIDX"
]
registerer.set_aws_ecr(docker=False, tags=tags) 

> Docker Build \
> ECR에 docker image를 push 합니다. \
> solution_metadata.yaml에 container uri를 넣어줍니다.

In [ ]:
# docker build & push
registerer.build_docker()
registerer.docker_push()
registerer.set_container_uri() 

----

#### **STEP-5**. Training 에 사용될 User Parameters 설정

> solution_metadata.yaml에 사용자 파라미터 및 artifacts 저장 경로를 넣어줍니다. 

In [ ]:
## candiate parameter 입력
registerer.set_candidate_parameters() 
## artifact 저장 경로 지정
registerer.set_artifacts_uri()

----

#### **STEP-6**. Training 에 사용될 Cloud resource 선택

> 컴퓨팅 자원을 선택합니다. (default: standard)

In [ ]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
# [참고] 현재 (release-2.1.3) 'standard'만 지원합니다. 
train_resource = 'standard'

#----------------------------------------#

# 클라우드 컴퓨팅 리소스 선택 
registerer.set_resource(train_resource)

----

#### **STEP-7**. Inference 용 Sample Data 등록

In [ ]:
###################
pipeline = "inference"
###################
# inference pipeline을 solution_metadata.yaml에 추가하고, 현재 registerer의 pipeline type을 inference로 변경 
registerer.append_pipeline(pipeline)
# s3 데이터 업로드
# 이전에 있던 데이터는 지워집니다
registerer.s3_access_check('data')
registerer.s3_upload_data()

registerer.s3_access_check('artifacts')
registerer.s3_upload_artifacts()


----

#### **STEP-8**. Inference 용 Docker Container 제작

> DOCKERFILE을 셋팅합니다. \
> AWS ECR에 docker 등록을 위한 repository를 생성합니다. 

In [ ]:
# DOCKERFILE setting
registerer.set_docker_contatiner()

## ECR 등록
## buildah login 실행 (docker in docker) 
tags = [
    "Key=Company,Value=LGE",
    "Key=Owner,Value=IC360",
    "Key=HQ,Value=CDO",
    "Key=Division,Value=CDO",
    "Key=Infra Region,Value=KIC",
    "Key=Service Mode,Value=DE",
    "Key=Cost Type,Value=COMPUTING",
    "Key=Project,Value=CIS",
    "Key=Sub Project,Value=CISM",
    "Key=System,Value=AIDX"
]
registerer.set_aws_ecr(docker=False, tags=tags) 

> Build docker \
> AWS ECR에 docker 이미지를 push 합니다. \
> solution_metadata.yaml에 container uri를 넣어줍니다.

In [ ]:
# docker build & push
registerer.build_docker()
registerer.docker_push()
registerer.set_container_uri() 

----

#### **STEP-9**. Inference 용 User Parameters 제작

> solution_metadata.yaml에 사용자 파라미터, artifacts 경로 등을 넣어줍니다. \
> solution_metadata.yaml에 model 경로를 넣어줍니다. <u>(추론 시에만 필요합니다.)</u>

In [ ]:
registerer.set_candidate_parameters()
registerer.set_artifacts_uri()
registerer.set_model_uri() # 참고: model은 train artifacts 경로에 존재

----

#### **STEP-10**. Inference 용 Resource 선택

> 컴퓨팅 자원을 선택합니다.

In [ ]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
# [참고] 현재 (release-2.1.3) 'standard'만 지원합니다. 
inference_resource = 'standard'

#----------------------------------------#

registerer.set_resource(inference_resource)

----

#### **STEP-11**. Description 추가

> solution_metadata.yaml에 description을 추가합니다.

In [ ]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
user_description ={
    'title': "240118 ws test", 
        
    'overview': "AI Advisor Test",

    'input_data': "Test input data",
    
    'output_data': "Test output data",

    'user_parameters': "Test params",
    
    'algorithm': "Tabular Classification"
}
#----------------------------------------#

registerer.set_description(user_description)

----

#### **STEP-12**. AI Solution & Instance 등록 !!

> AI Solution 등록을 진행합니다.

In [ ]:
# [참고] release-2.1.3에서는 wrangler 미지원 
# wrangler 정보 등록 
registerer.set_wrangler()

In [ ]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

# EdgeConductor관련 정보 등록
edgeconductor_interface = {
            # 모델 재학습 시 re-labeling이 가능한 컨텐츠 인지 
            'support_labeling': True,
            # EdgeConductor UI에서 보여질 추론 결과 데이터 타입 
            'inference_result_datatype': 'table', # 'image' / 'table'
            # 모델 재학습 시 입력으로 들어가는 데이터 타입 
            'train_datatype': 'table' # 'image' / 'table'
        }

#----------------------------------------#

registerer.set_edge(edgeconductor_interface)

> Solution 등록

In [ ]:
# 솔루션 등록 !! 
registerer.register_solution()

> Solution instance 등록

In [ ]:
# 솔루션 instance 등록 !! 
registerer.register_solution_instance()

In [ ]:
# Stream 생성 및 실행은 EdgeConductor UI에서 진행해주세요. 
# registerer.register_stream()
# registerer.request_run_stream()

----